In [4]:
import numpy as np
import matplotlib.pyplot as plt

# 신경망 학습

## 데이터에서 학습

### 데이터 주도 학습
기계학습의 중심에는 `데이터` 존재

### 훈련, 시험 데이터
나누는 이유: 원하는 것이 범용적으로 사용할 수 있는 모델이기 때문이다. 즉, `범용 능력`을 제대로 평가하기 위해 두 개의 데이터로 나눈다.

`오버피팅`: 한 데이터셋에만 지나치게 최적화된 상태

## 손실 함수
신경망에서 '하나의 지표'를 기준으로 최적의 매개변수 값을 탐색함. 이 때 사용되는 지표가 `손실 함수(loss function)`이다.

### 오차제곱합
$$E = \frac{1}{2}\sum_{k}(y_k - t_k)^2$$
변수 | 설명
:---:|----
k    | 데이터 차원 수
$y_k$|신경망의 출력
$t_k$|정답 레이블

In [3]:
# 오차제곱합
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

### 교차 엔트로피 오차
$$E=-\sum_{k}t_k\log y_k$$
여기서 $\log$는 자연로그 ($\log_e$)다.

In [5]:
# 교차 엔트로피 오차
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

delta는 np.log에 0이 들어가 -inf값이 되지 않게 한다.

### 미니 배치 학습
모든 훈련 데이터에 대한 손실 함수의 합
* 교차 엔트로피 오차
$$E=-\frac{1}{N} \sum_{n} \sum_{k} t_{nk}\log y_{nk}$$

모든 데이터를 대상으로 손실 함수의 합을 구하려면 시간이 오래 걸린다. 이 경우 데이터 일부를 추려 전체의 '근사치'로 이용하는 방법을 이용할 수 있다. 신경망 학습에서는 일부만 골라 학습을 수행하는 것을 `미니배치 학습`이라고 한다. 데이터의 일부를 `미니배치`라고 한다.

In [6]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [7]:
print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [8]:
# 훈련 데이터에서 무작위로 10장만 빼내기
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

### (배치용) 교차 엔트로피 오차 구현

In [13]:
# 정답 레이블이 one-hot encoding인 경우
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [18]:
# 정답 레이블이 one-hot encoding이 아닌 경우
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

### 손실 함수 설정 이유
정확도를 지표로 삼으면 안되는 이유: 정확도에는 연속적인 변화가 없기 때문에 미분이 대부분의 장소에서 0이 되기 때문
손실 함수를 지표로 삼는 이유: 매개변수의 값이 조금만 변해도 이에 반응하여 연속적으로 손실 함수의 값이 변하며, 미분 값도 연속적으로 변하기 때문

## 수치 미분